In [16]:
import pandas as pd
# gpt
df = pd.read_csv('GPT_discription/UCF_101_gpt_Class_discription_new.csv')
df

,SNo,Class Name,GPT3 discription
0,1,ApplyEyeMakeup,\n\nApplyEyeMakeup is an action performed by h...
1,2,ApplyLipstick,\n\nApplyLipstick is an action performed by hu...
2,3,Archery,\n\nArchery is an action performed by humans w...
3,4,BabyCrawling,\n\nBaby crawling is an action performed by ba...
4,5,BalanceBeam,\n\nBalanceBeam is an action performed by huma...
...,...,...,...
96,97,VolleyballSpiking,\n\nVolleyball spiking is an action performed ...
97,98,WalkingWithDog,\n\nWalking with a dog is an action performed ...
98,99,WallPushups,\n\nWall pushups are a bodyweight exercise tha...
99,100,WritingOnBoard,\n\nWritingOnBoard is an action performed by a...


In [3]:
# label_list
label_df = pd.read_csv('lists/ucf_labels.csv')
label_df

,id,name
0,0,ApplyEyeMakeup
1,1,ApplyLipstick
2,2,Archery
3,3,BabyCrawling
4,4,BalanceBeam
...,...,...
96,96,VolleyballSpiking
97,97,WalkingWithDog
98,98,WallPushups
99,99,WritingOnBoard


In [15]:
import os
# read txt
with open('dataset_splits/UCF/Zero_shot/test.txt', 'r') as f:
    test_txt = f.readlines()
test_txt = [x.strip() for x in test_txt]
test_txt

['/cluster/datastore/shahzaa/data_for_action_clip/UCF/v_SoccerPenalty_g04_c03 70 84',
 '/cluster/datastore/shahzaa/data_for_action_clip/UCF/v_JumpingJack_g06_c05 70 46',
 '/cluster/datastore/shahzaa/data_for_action_clip/UCF/v_CricketShot_g04_c06 77 23',
 '/cluster/datastore/shahzaa/data_for_action_clip/UCF/v_CricketShot_g03_c04 101 23',
 '/cluster/datastore/shahzaa/data_for_action_clip/UCF/v_Bowling_g05_c05 118 15',
 '/cluster/datastore/shahzaa/data_for_action_clip/UCF/v_SoccerPenalty_g06_c03 65 84',
 '/cluster/datastore/shahzaa/data_for_action_clip/UCF/v_CricketShot_g07_c04 118 23',
 '/cluster/datastore/shahzaa/data_for_action_clip/UCF/v_ApplyEyeMakeup_g07_c02 168 0',
 '/cluster/datastore/shahzaa/data_for_action_clip/UCF/v_CuttingInKitchen_g03_c03 266 24',
 '/cluster/datastore/shahzaa/data_for_action_clip/UCF/v_Shotput_g02_c05 94 78',
 '/cluster/datastore/shahzaa/data_for_action_clip/UCF/v_SoccerPenalty_g03_c05 70 84',
 '/cluster/datastore/shahzaa/data_for_action_clip/UCF/v_WritingOnB

In [2]:
import os
base_dir = '/root/wj/EZ_CLIP/configs'
dataset = ['HARDVS','PAF','DVS128Gesture','SeAct']
file = ['{}_train.yaml','{}_zero_shot_testing.yaml']
for d in dataset:
    d_path = os.path.join(base_dir,d)
    os.makedirs(d_path)
    for f in file:
        f_path = os.path.join(d_path,f.format(d))
        # make file
        os.mknod(f_path)

In [8]:
base_dir = '/root/wj/EZ_CLIP/lists'
file = '{}_labels.csv'
for d in  dataset:
    d_path = os.path.join(base_dir,file.format(d))
    os.mknod(d_path)

/root/wj/EZ_CLIP/lists/HARDVS_labels.csv
/root/wj/EZ_CLIP/lists/PAF_labels.csv
/root/wj/EZ_CLIP/lists/DVS128Gesture_labels.csv


In [9]:
base_dir = '/root/wj/EZ_CLIP/GPT_discription'
file = '{}_gpt_Class_discription_new.csv'
for d in  dataset:
    d_path = os.path.join(base_dir,file.format(d))
    os.mknod(d_path)

In [11]:
base_dir = '/root/wj/EZ_CLIP/dataset_splits'
relative_dir = 'Zero-shot'
file = ['train.txt','val.txt']
for d in dataset:
    for f in file:
        d_path = os.path.join(base_dir,d,relative_dir,f)
        # make dir and make file
        os.makedirs(os.path.dirname(d_path), exist_ok=True)
        os.mknod(d_path)


In [3]:
import os
import yaml

config_dir = '/root/wj/EZ_CLIP/configs'
dataset = ['HARDVS', 'PAF', 'DVS128Gesture']
config_file_templates = ['{}_train.yaml', '{}_zero_shot_testing.yaml']
relative_dirs = ['dataset_splits', 'lists', 'GPT_discription']
relative_files = ['{}/Zero-shot/val.txt', '{}_labels.csv', '{}_gpt_Class_discription_new.csv']
relative_keys = ['val_list', 'label_list', 'gpt_discription']

for d in dataset:
    for config_template in config_file_templates:
        config_file = os.path.join(config_dir, d, config_template.format(d))
        
        # Read existing YAML content
        if os.path.exists(config_file):
            with open(config_file, 'r') as infile:
                config_data = yaml.safe_load(infile) or {}
        else:
            config_data = {}

        # Modify or add new entries
        for i in range(3):
            path = os.path.join(relative_dirs[i], relative_files[i].format(d))
            key = relative_keys[i]
            config_data['data'][key] = path
        if config_template == '{}_train.yaml':
            config_data['data']['train_list'] = os.path.join(relative_dirs[0], d, 'Zero-shot/train.txt')
        config_data['data']['dataset'] = d
        config_data['network']['type'] = 'CLIP_T_Adapter'
        config_data['weight_save_dir'] = '/root/autodl-tmp/EZ_CLIP_events'
        config_data['training_name'] = f"{config_template.format(d).replace('.yaml', '')}_vit_16_8_frame"
        config_data['T_Adapter'] = True
        config_data['prompt']['use']=False
        # Write back to YAML file
        with open(config_file, 'w') as outfile:
            yaml.dump(config_data, outfile, default_flow_style=False)

print("Configuration files have been updated successfully.")


Configuration files have been updated successfully.
